In [1]:
import os
import hashlib
from os import path, makedirs
import shutil
import logging
import time
import pandas as pd
import tempfile
import json
import requests
import base64
from modules.data_processing import categorize_files_by_template, pdf_to_image_paths
from modules.arrange_conf_md import get_log_directory_md, get_output_directory_md, get_data_paths
from modules.batch_processing import split_jsonl_file, send_batch_requests
from config import MODEL_gpt_4, prompt_md
from multiprocessing import current_process

/home/gaye/miniconda3/envs/vt2/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
directory = 'vrdu2/registration-form/few_shot-splits/'
# directory = 'VT2/vrdu2/ad-buy-form/few_shot-splits/'

# Arrange logging and output dirs accordıng to model and form type
model = MODEL_gpt_4  #MODEL_gpt_4 #  or 
log_directory = get_log_directory_md(directory, model) 
base_output_directory = get_output_directory_md(directory, model)

print("log_directory: ", log_directory)
print("base_output_directory: ", base_output_directory)

# Arrange folder and dataset dirs based on the provided main dir
folder_path, dataset_path, dtype = get_data_paths(directory)

print("folder_path:", folder_path)
print("dataset_path:", dataset_path)
print("dtype:", dtype)

log_file_path = path.join(log_directory, f"experiment_{current_process().pid}.log")
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[
                        logging.FileHandler(log_file_path),
                        logging.StreamHandler()
                    ])

# Example logging
logging.info("Logging setup complete.")

2025-02-24 11:42:24,197 - INFO - Logging setup complete.


log_directory:  gpt4_Markdown_outputs/reg/
base_output_directory:  gpt4_Markdown_outputs/reg/
folder_path: vrdu2/registration-form/main/pdfs
dataset_path: vrdu2/registration-form/main/dataset.jsonl.gz
dtype: reg


In [3]:
import itertools

sample_size = 40
level_types = ["UTL", "STL"]

combinations = []
experiment_id = 0

for level_type in level_types:
    combination = {
        'experiment_id': experiment_id,
        'dataset': "vrdu",
        'sample_size': sample_size,
        'level_type': level_type
    }
    combinations.append(combination)
    experiment_id += 1
    
# Creating a DataFrame
df = pd.DataFrame(combinations)

# Saving to CSV
csv_file_path = path.join(base_output_directory, 'all_experiment_combinations.csv')
df.to_csv(csv_file_path, index=False)

print(f"CSV file has been saved to {csv_file_path}")

CSV file has been saved to gpt4_Markdown_outputs/reg/all_experiment_combinations.csv


In [14]:
def process_batch_responses_local(file_path, output_dir, all_filename_mappings):
    print("Processing file: ", file_path)
    
    with open(file_path, 'r', encoding='utf-8') as f:
        file_content = f.read()
    
    print("file_content: ", file_content[:500])  # Print the first 500 characters for inspection

    for line in file_content.splitlines():
        response = json.loads(line)
        custom_id = response['custom_id']
        message_content = response['response']['body']['choices'][0]['message']['content']

        parts = custom_id.split('_')
        experiment_id = parts[0]
        level_type = parts[1]
        template_type = parts[2]
        sanitized_filename = '_'.join(parts[3:-2])
        page_num = parts[-1]
        
        original_filename = all_filename_mappings.get(sanitized_filename, sanitized_filename).replace(".pdf", "")
        
        markdown_dir = os.path.join(output_dir, level_type, template_type)
        markdown_file_path = os.path.join(markdown_dir, f"{original_filename}.md")
        os.makedirs(markdown_dir, exist_ok=True)

        with open(markdown_file_path, 'a', encoding='utf-8') as f:
            f.write(message_content + '\n\n')

        logging.info(f"Markdown file updated at {markdown_file_path}")
        print(f"Markdown file updated at {markdown_file_path}")

def retrieve_and_process_batch_results(response_dir, output_dir, all_filename_mappings):
     for filename in os.listdir(response_dir):
        if filename.endswith(".jsonl"):
            file_path = os.path.join(response_dir, filename)
            process_batch_responses_local(file_path, output_dir, all_filename_mappings)

def sanitize_filename(filename):
    sanitized = filename.replace(" ", "_").replace(",", "").replace("(", "").replace(")", "").replace("&", "").replace("'", "").replace("\"", "").replace(".", "").replace("/", "").strip()
    return sanitized, filename

def generate_custom_id(experiment_id, level_type, template_type, filename, page_num, task_counter):
    # Create a hash to ensure the custom_id is unique
    unique_string = f"{experiment_id}_{level_type}_{template_type}_{filename}_{page_num}_{task_counter}"
    unique_hash = hashlib.md5(unique_string.encode()).hexdigest()
    return f"{experiment_id}_{level_type}_{template_type}_{filename}_{page_num}_{unique_hash}"
    
def perform_experiment(sample_size, experiment_id, output_file, **kwargs):
    logging.info(f"Starting experiment with ID {experiment_id}")
    print(f"Starting experiment with ID {experiment_id}")
    try:
        level_type = kwargs.get('level_type', 'STL')
        template_types = ['Amendment', 'Dissemination', 'Short-Form']
        logging.debug(f"Categorizing files by template for experiment ID {experiment_id}")

        categorized_files = categorize_files_by_template(os.path.join(directory, 'prompts'))
        logging.debug(f"Loaded categorized files for level_type {level_type}")

        task_counter = 0  
        existing_custom_ids = set()
        filename_mapping = {}
        
        for template_type in template_types:
            print("s---------------Template type: ", template_type)
            print("s---------------Level type: ", level_type) 
            logging.info(f"Processing template type: {template_type}")
            
            files_list = list(categorized_files[level_type][template_type])
            sample_files = files_list[:sample_size]
            logging.debug(f"Loaded sample files for template type {template_type}: {sample_files}")

            for filename in sample_files:
                if filename.endswith(".pdf"):
                    logging.info(f"Experiment {experiment_id}: Processing file {filename}")
                    print(f"Experiment {experiment_id}: Processing file {filename}")

                    success = False
                    retry_count = 0
                    max_retries = 10
                    sanitized_filename, original_filename = sanitize_filename(filename)
                    filename_mapping[sanitized_filename] = original_filename 

                    while not success and retry_count < max_retries:
                        try:
                            pdf_path = os.path.join(folder_path, filename)
                            with tempfile.TemporaryDirectory() as temp_pdf_dir:
                                temp_pdf_path = os.path.join(temp_pdf_dir, filename)
                                shutil.copy(pdf_path, temp_pdf_path)

                                images = pdf_to_image_paths(temp_pdf_path, temp_pdf_dir)
                                if not images:
                                    logging.error(f"No images generated from PDF: {pdf_path}")
                                    print(f"No images generated from PDF: {pdf_path}")
                                    break

                                for page_num, image_path in enumerate(images):
                                    if image_path.startswith("http"):
                                        image_url = image_path
                                    else:
                                        image_url = f"data:image/jpeg;base64,{encode_image(image_path)}"
                                        
                                    custom_id = generate_custom_id(experiment_id, level_type, template_type, sanitized_filename, page_num, task_counter)

                                    while custom_id in existing_custom_ids:
                                        task_counter += 1
                                        custom_id = generate_custom_id(experiment_id, level_type, template_type, sanitized_filename, page_num, task_counter)

                                    existing_custom_ids.add(custom_id)
                                    task_counter += 1
                                    
                                    request_body = {
                                        "model": MODEL,
                                        "temperature": 0.0,
                                        "messages": [
                                            {
                                                "role": "system",
                                                "content": "You are a helpful assistant.",
                                            },
                                            {
                                                "role": "user", 
                                                "content": [
                                                    {"type": "text", "text": prompt_md},
                                                    {
                                                        "type": "image_url",
                                                        "image_url": {"url": image_url}
                                                    }
                                                ]
                                            },
                                        ],
                                    }
                                    task = {
                                        "custom_id": custom_id,
                                        "method": "POST",
                                        "url": "/v1/chat/completions",
                                        "body": request_body,
                                    }
                                    with open(output_file, "a") as file:
                                        file.write(json.dumps(task) + "\n")

                                success = True
                        except requests.exceptions.HTTPError as e:
                            if e.response.status_code == 429:
                                retry_count += 1
                                logging.warning(f"Rate limit reached for file {filename}. Retrying... ({retry_count}/{max_retries})")
                                print(f"Rate limit reached for file {filename}. Retrying... ({retry_count}/{max_retries})")
                                time.sleep(2 ** retry_count)
                            else:
                                logging.error(f"HTTP error while processing file {filename}: {e}")
                                print(f"HTTP error while processing file {filename}: {e}")
                                break  # Exit loop on non-rate limit HTTP errors
                        except Exception as e:
                            logging.error(f"An error occurred while processing file {filename}: {e}")
                            print(f"An error occurred while processing file {filename}: {e}")
                            break  # Exit loop on other errors

    except Exception as e:
        logging.error(f"Error during experiment {experiment_id}: {e}")
        print(f"Error during experiment {experiment_id}: {e}")
        return pd.DataFrame()

    return filename_mapping

In [16]:
def run_experiment(params, output_file):
    sample_size, kwargs, experiment_id = params

    logging.info(f"Preparing to run experiment ID {experiment_id}")
    print(f"Preparing to run experiment ID {experiment_id}")
    try:
        filename_mapping = perform_experiment(sample_size, experiment_id, output_file, **kwargs)
        return filename_mapping
    except Exception as e:
        logging.error(f"An error occurred in experiment {experiment_id}: {e}")
        print(f"An error occurred in experiment {experiment_id}: {e}")

def test_experiment_serially(output_file):
    all_filename_mappings = {}
    for i, params in enumerate(experiment_params):
        print(f"Testing Experiment {i} with params: {params}")
        try:
            filename_mapping = run_experiment(params, output_file)
            all_filename_mappings.update(filename_mapping)  # Collect all mappings
            print("Experiment completed successfully.")
        except Exception as e:
            print(f"Experiment resulted in an exception: {e}")
    return all_filename_mappings

In [17]:
if __name__ == "__main__":
    experiment_params = [(exp['sample_size'], {'level_type': exp['level_type']}, exp['experiment_id']) for exp in combinations]
    output_file = os.path.join(base_output_directory, 'batch_requests.jsonl')
    
    all_filename_mappings = test_experiment_serially(output_file)
    

2024-08-09 10:16:59,474 - INFO - Preparing to run experiment ID 0
2024-08-09 10:16:59,477 - INFO - Starting experiment with ID 0
2024-08-09 10:16:59,480 - INFO - Processing template type: Amendment
2024-08-09 10:16:59,481 - INFO - Experiment 0: Processing file 19790101_Representative of the Turkish Republic of Northern Cyprus_Amendment_Amendment.pdf


Testing Experiment 0 with params: (40, {'level_type': 'UTL'}, 0)
Preparing to run experiment ID 0
Starting experiment with ID 0
s---------------Template type:  Amendment
s---------------Level type:  UTL
Experiment 0: Processing file 19790101_Representative of the Turkish Republic of Northern Cyprus_Amendment_Amendment.pdf


2024-08-09 10:17:00,287 - INFO - Converted and resized image saved to /tmp/tmpqfxqyzlc/page_1.jpg with final quality 85


Converted and resized image saved to /tmp/tmpqfxqyzlc/page_1.jpg with final quality 85


2024-08-09 10:17:00,613 - INFO - Converted and resized image saved to /tmp/tmpqfxqyzlc/page_2.jpg with final quality 85
2024-08-09 10:17:00,615 - INFO - Converted PDF /tmp/tmpqfxqyzlc/19790101_Representative of the Turkish Republic of Northern Cyprus_Amendment_Amendment.pdf to images: ['/tmp/tmpqfxqyzlc/page_1.jpg', '/tmp/tmpqfxqyzlc/page_2.jpg']
2024-08-09 10:17:00,624 - INFO - Experiment 0: Processing file 20090930_Garvey, Schubert _ Barer_Amendment_Amendment.pdf


...

Converted and resized image saved to /tmp/tmp08vjyle7/page_2.jpg with final quality 85
Converted PDF /tmp/tmp08vjyle7/20100909_Gephardt Group Government Affairs, LLC_Messmer, Michael_Short-Form.pdf to images: ['/tmp/tmp08vjyle7/page_1.jpg', '/tmp/tmp08vjyle7/page_2.jpg']
Experiment 1: Processing file 20170112_Podesta Group, Inc._Thompson, Dana_Short-Form.pdf


2024-08-09 10:20:55,724 - INFO - Converted and resized image saved to /tmp/tmpzvf8rg1x/page_1.jpg with final quality 85


Converted and resized image saved to /tmp/tmpzvf8rg1x/page_1.jpg with final quality 85


2024-08-09 10:20:56,089 - INFO - Converted and resized image saved to /tmp/tmpzvf8rg1x/page_2.jpg with final quality 85
2024-08-09 10:20:56,091 - INFO - Converted PDF /tmp/tmpzvf8rg1x/20170112_Podesta Group, Inc._Thompson, Dana_Short-Form.pdf to images: ['/tmp/tmpzvf8rg1x/page_1.jpg', '/tmp/tmpzvf8rg1x/page_2.jpg']


Converted and resized image saved to /tmp/tmpzvf8rg1x/page_2.jpg with final quality 85
Converted PDF /tmp/tmpzvf8rg1x/20170112_Podesta Group, Inc._Thompson, Dana_Short-Form.pdf to images: ['/tmp/tmpzvf8rg1x/page_1.jpg', '/tmp/tmpzvf8rg1x/page_2.jpg']
Experiment completed successfully.


In [ ]:
batch_file_paths = split_jsonl_file(output_file, max_size=209715200, max_lines=500)
batch_ids = send_batch_requests(batch_file_paths)
print(f"Batch IDs: {batch_ids}")

In [18]:
output_dir = os.path.join(base_output_directory, "outputs")
response_dir = os.path.join(base_output_directory, "responses")

retrieve_and_process_batch_results(response_dir, output_dir, all_filename_mappings)

2024-08-09 10:20:56,117 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/UTL/Amendment/20090930_Garvey, Schubert _ Barer_Amendment_Amendment.md
2024-08-09 10:20:56,119 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/UTL/Amendment/20090930_Garvey, Schubert _ Barer_Amendment_Amendment.md
2024-08-09 10:20:56,120 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/UTL/Amendment/19760201_Italian Government Tourist Board (ENIT), Los Angeles_Amendment_Amendment.md
2024-08-09 10:20:56,121 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/UTL/Amendment/19760201_Italian Government Tourist Board (ENIT), Los Angeles_Amendment_Amendment.md
2024-08-09 10:20:56,122 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/UTL/Amendment/19760201_Italian Government Tourist Board (ENIT), Los Angeles_Amendment_Amendment.md
2024-08-09 10:20:56,123 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/UTL/Amend

Processing file:  gpt4_Markdown_outputs/reg/responses/batch_NEmKL4xiVOreeI8WcArrCeEw_output.jsonl
file_content:  {"id": "batch_req_Uwt959fNCd1UcoqEcNxOfuYN", "custom_id": "0_UTL_Amendment_20090930_Garvey_Schubert___Barer_Amendment_Amendmentpdf_0_b363b55d09a1e1f6b18aa4b5cc2d3200", "response": {"status_code": 200, "request_id": "1fa9640a4b3a5d0c5dbff2d3529a7ca8", "body": {"id": "chatcmpl-9uGUsquNbVlp85YOEot62zNk4fGdJ", "object": "chat.completion", "created": 1723196810, "model": "gpt-4o-2024-05-13", "choices": [{"index": 0, "message": {"role": "assistant", "content": "```markdown\n1. Name of Registrant\n    
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/UTL/Amendment/20090930_Garvey, Schubert _ Barer_Amendment_Amendment.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/UTL/Amendment/20090930_Garvey, Schubert _ Barer_Amendment_Amendment.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/UTL/Amendment/19760201_Italian Government Tourist Board (ENIT), Los

2024-08-09 10:20:56,292 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Amendment/20010820_Arnold _ Porter Kaye Scholer, LLP_Amendment_Amendment.md
2024-08-09 10:20:56,293 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Amendment/20170901_Bryan Cave, LLP_Amendment_Amendment.md
2024-08-09 10:20:56,293 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Amendment/20170901_Bryan Cave, LLP_Amendment_Amendment.md
2024-08-09 10:20:56,294 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Amendment/20030714_Arnold _ Porter Kaye Scholer, LLP_Amendment_Amendment.md
2024-08-09 10:20:56,294 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Amendment/20030714_Arnold _ Porter Kaye Scholer, LLP_Amendment_Amendment.md
2024-08-09 10:20:56,295 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Amendment/19950929_Hogan Lovells US LLP_Amendment_Amendment.md
2024-08-09 10:20:

Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Amendment/20010820_Arnold _ Porter Kaye Scholer, LLP_Amendment_Amendment.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Amendment/20010820_Arnold _ Porter Kaye Scholer, LLP_Amendment_Amendment.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Amendment/20170901_Bryan Cave, LLP_Amendment_Amendment.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Amendment/20170901_Bryan Cave, LLP_Amendment_Amendment.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Amendment/20030714_Arnold _ Porter Kaye Scholer, LLP_Amendment_Amendment.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Amendment/20030714_Arnold _ Porter Kaye Scholer, LLP_Amendment_Amendment.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Amendment/19950929_Hogan Lovells US LLP_Amendment_Amendment.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Amendment/199

2024-08-09 10:20:56,317 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Dissemination/19920301_JETRO, Los Angeles_Dissemination Report_Dissemination Report.md
2024-08-09 10:20:56,318 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Dissemination/19920301_JETRO, Los Angeles_Dissemination Report_Dissemination Report.md
2024-08-09 10:20:56,319 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Dissemination/19920701_JETRO, Los Angeles_Dissemination Report_Dissemination Report.md
2024-08-09 10:20:56,319 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Dissemination/19920701_JETRO, Los Angeles_Dissemination Report_Dissemination Report.md
2024-08-09 10:20:56,320 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Dissemination/19950110_JETRO, New York_Dissemination Report_Dissemination Report.md
2024-08-09 10:20:56,320 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outp

Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Dissemination/19920301_JETRO, Los Angeles_Dissemination Report_Dissemination Report.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Dissemination/19920301_JETRO, Los Angeles_Dissemination Report_Dissemination Report.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Dissemination/19920701_JETRO, Los Angeles_Dissemination Report_Dissemination Report.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Dissemination/19920701_JETRO, Los Angeles_Dissemination Report_Dissemination Report.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Dissemination/19950110_JETRO, New York_Dissemination Report_Dissemination Report.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Dissemination/19930601_International Registries, Inc._Dissemination Report_Dissemination Report.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Dissemination/19930601_Inter

2024-08-09 10:20:56,492 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Short-Form/20090409_Daniel J. Edelman, Inc._Segall, Jules Peter_Short-Form.md
2024-08-09 10:20:56,493 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Short-Form/20140507_Ketchum Inc. NY_Amorosi, Alexandra Scott_Short-Form.md
2024-08-09 10:20:56,494 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Short-Form/20140507_Ketchum Inc. NY_Amorosi, Alexandra Scott_Short-Form.md
2024-08-09 10:20:56,495 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Short-Form/20090522_Invest Northern Ireland_Murray, Aengus_Short-Form.md
2024-08-09 10:20:56,495 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Short-Form/20090522_Invest Northern Ireland_Murray, Aengus_Short-Form.md
2024-08-09 10:20:56,496 - INFO - Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Short-Form/20160731_Burson-Marsteller, LLC (Washin

Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Short-Form/20090409_Daniel J. Edelman, Inc._Segall, Jules Peter_Short-Form.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Short-Form/20140507_Ketchum Inc. NY_Amorosi, Alexandra Scott_Short-Form.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Short-Form/20140507_Ketchum Inc. NY_Amorosi, Alexandra Scott_Short-Form.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Short-Form/20090522_Invest Northern Ireland_Murray, Aengus_Short-Form.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Short-Form/20090522_Invest Northern Ireland_Murray, Aengus_Short-Form.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Short-Form/20160731_Burson-Marsteller, LLC (Washington, DC)_Jarnagan, Stephanie_Short-Form.md
Markdown file updated at gpt4_Markdown_outputs/reg/outputs/STL/Short-Form/20160731_Burson-Marsteller, LLC (Washington, DC)_Jarnagan, Stephanie_Short-Form.

## Check Consistency of Files

In [19]:
import os

def check_output_files(output_dir):
    levels = ['STL', 'UTL']
    template_types = ['Amendment', 'Dissemination', 'Short-Form']
    
    level_files = {level: {template: set() for template in template_types} for level in levels}
    
    # Collect file names for each level and template type
    for level in levels:
        for template in template_types:
            template_dir = os.path.join(output_dir, level, template)
            if os.path.exists(template_dir):
                level_files[level][template] = set(os.listdir(template_dir))
            else:
                print(f"Directory does not exist: {template_dir}")

    all_checks_passed = True

    # Compare files between corresponding template types in different levels
    for template in template_types:
        stl_files = level_files['STL'][template]
        utl_files = level_files['UTL'][template]
        
        if stl_files != utl_files:
            all_checks_passed = False
            print(f"Discrepancy found in template type '{template}':")
            print(f"Files in STL: {stl_files}")
            print(f"Files in UTL: {utl_files}")
        else:
            print(f"Template type '{template}' matches between STL and UTL.")

    if all_checks_passed:
        print("All checks passed. The files in 'outputs' directory are consistent.")
    else:
        print("Some discrepancies were found. Please check the details above.")

# Usage example
output_dir = 'gpt4_Markdown_outputs/reg/outputs'  # Replace with your actual output directory path
check_output_files(output_dir)


Template type 'Amendment' matches between STL and UTL.
Template type 'Dissemination' matches between STL and UTL.
Template type 'Short-Form' matches between STL and UTL.
All checks passed. The files in 'outputs' directory are consistent.
